In [ ]:
import random
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import eli5
from eli5.sklearn import PermutationImportance
import matplotlib.pyplot as plt

In [ ]:
Base_data_merge=pd.read_csv('E:/Documents/Yagna/prep_data/Base_data_merge.csv')

In [ ]:
Base_data_merge.drop(['Total_Price', 'Count_pro_ser', 'Cat_pro_ser', 'CountForCat_pro_ser',
       'Quantity_log', 'CountForCat_pro_ser_bin', 'Total_Price_log',
       'Product_price_log', 'services_price_log'], axis=1, inplace=True)


In [ ]:
#Base_data_merge.head()

In [ ]:
#Base_data_merge.info()

In [ ]:
Base_data_merge.columns

In [ ]:
 Base_data_merge_ltd=Base_data_merge[['Existing_Contract_Number', 'Sales_Stage',
       'Expiring_Contract_End_Date', 
       'Renewal_Contract_Duration', 'EA_Segment', 'Country_Name', 'State',
       'X__Original_Opty_Val_Annualized',
       'PFST_Geo', 'PFST_Region', 'Batch_Type', 'Sub-Batch_Type',
       'First_Renewal', 'CON_Existing_Contract', 'Diff_inDays', 'RenewalType',
       'CUS_On_Time_percent', 'CUS_Early_percent', 'CUS_Delayed_percent',
       'CON_On_Time_percent', 'CON_Early_percent', 'CON_Delayed_percent',
       'mark_1_if_renewal', 'CON_number_renewal', 'CUS_number_renewal',
       'CON_Renewal_Contract_Duration_0.05_percentile',
       'CON_Renewal_Contract_Duration_0.1_percentile',
       'CON_Renewal_Contract_Duration_0.25_percentile',
       'CON_Renewal_Contract_Duration_0.5_percentile',
       'CON_Renewal_Contract_Duration_0.75_percentile',
       'CON_Renewal_Contract_Duration_0.9_percentile',
       'CON_Renewal_Contract_Duration_0.95_percentile',
       'CON_Renewal_Contract_Duration_0.99_percentile',
       'CON_Diff_inDays_0.05_percentile', 'CON_Diff_inDays_0.1_percentile',
       'CON_Diff_inDays_0.25_percentile', 'CON_Diff_inDays_0.5_percentile',
       'CON_Diff_inDays_0.75_percentile', 'CON_Diff_inDays_0.9_percentile',
       'CON_Diff_inDays_0.95_percentile', 'CON_Diff_inDays_0.99_percentile',
       'CUS_Renewal_Contract_Duration_0.05_percentile',
       'CUS_Renewal_Contract_Duration_0.1_percentile',
       'CUS_Renewal_Contract_Duration_0.25_percentile',
       'CUS_Renewal_Contract_Duration_0.5_percentile',
       'CUS_Renewal_Contract_Duration_0.75_percentile',
       'CUS_Renewal_Contract_Duration_0.9_percentile',
       'CUS_Renewal_Contract_Duration_0.95_percentile',
       'CUS_Renewal_Contract_Duration_0.99_percentile',
       'CUS_Diff_inDays_0.05_percentile', 'CUS_Diff_inDays_0.1_percentile',
       'CUS_Diff_inDays_0.25_percentile', 'CUS_Diff_inDays_0.5_percentile',
       'CUS_Diff_inDays_0.75_percentile', 'CUS_Diff_inDays_0.9_percentile',
       'CUS_Diff_inDays_0.95_percentile', 'CUS_Diff_inDays_0.99_percentile',
       'is_current_renewal_First_Renewal_', 'last_renewal']]

In [ ]:
Base_data_merge_ltd['Sales_Stage']=Base_data_merge_ltd['Sales_Stage'].map({'06 - Closed Won':0, '07 - Closed Lost':1})
Base_data_merge_ltd.Expiring_Contract_End_Date=pd.to_datetime(Base_data_merge_ltd.Expiring_Contract_End_Date, format='%Y-%m-%d')
Base_data_merge_ltd.Renewal_Contract_Duration=Base_data_merge_ltd.Renewal_Contract_Duration.astype(float)
Base_data_merge_ltd.X__Original_Opty_Val_Annualized=pd.cut(Base_data_merge_ltd.X__Original_Opty_Val_Annualized, 20).cat.codes
Base_data_merge_ltd.Renewal_Contract_Duration=pd.cut(Base_data_merge_ltd.Renewal_Contract_Duration, 20).cat.codes
Base_data_merge_ltd.First_Renewal=Base_data_merge_ltd.First_Renewal.astype(str).map({'0':False, '1':True, '2':True})

In [ ]:
#Base_data_merge_ltd.EA_Segment.unique()

In [ ]:
#Base_data_merge_ltd.columns

In [ ]:
Base_data_merge_ltd.loc[Base_data_merge_ltd.last_renewal=='Did not renew',['last_renewal']]=''

In [ ]:
Base_data_merge_ltd.drop('PFST_Region', axis=1, inplace=True)
Base_data_merge_ltd.drop('Expiring_Contract_End_Date', axis=1, inplace=True)    
Base_data_merge_ltd.drop('RenewalType', axis=1, inplace=True)   
Base_data_merge_ltd.drop('Sub-Batch_Type', axis=1, inplace=True)   
Base_data_merge_ltd.drop('mark_1_if_renewal', axis=1, inplace=True)   



In [ ]:
dum = pd.get_dummies(Base_data_merge_ltd[['EA_Segment','Country_Name','State','PFST_Geo','Batch_Type','last_renewal']], drop_first = True)

In [ ]:
Base_data_merge_ltd = pd.concat([Base_data_merge_ltd, dum], axis = 1)

# Now let's see the head of our dataframe.
#housing.head()
# Drop 'furnishingstatus' as we have created the dummies for it
Base_data_merge_ltd_1hot=Base_data_merge_ltd.drop(['EA_Segment','Country_Name','State','PFST_Geo','Batch_Type','last_renewal'], axis = 1)

#housing.head()

In [ ]:
#Base_data_merge_ltd.info()

In [ ]:
#Base_data_merge_ltd_null=Base_data_merge_ltd_1hot.isnull().sum()

In [ ]:
#Base_data_merge_ltd_1hot.loc[Base_data_merge_ltd_1hot>0]

In [ ]:
#Base_data_merge_ltd_clean.isnull().sum()

In [ ]:
Base_data_merge_ltd_clean=Base_data_merge_ltd_1hot.dropna()

In [ ]:
Base_data_merge_ltd_clean.shape

In [ ]:
Y = Base_data_merge_ltd_clean.Sales_Stage
X =Base_data_merge_ltd_clean.drop('Sales_Stage', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train_Existing_Contract_Number=X_train.Existing_Contract_Number                         
X_test_Existing_Contract_Number=X_test.Existing_Contract_Number   

X_train =X_train.drop('Existing_Contract_Number', axis=1)
X_test =X_test.drop('Existing_Contract_Number', axis=1)


In [ ]:
#X_train.
#X_testgroupby('ContractNumber').count()
print(y_train.count())
print(y_test.count())
print(y_train.sum())
print(y_test.sum())
print(y_train.sum()/y_train.count())
print(y_test.sum()/y_test.count())

In [ ]:
pipeline = Pipeline([('sca',MinMaxScaler()),
                     ('chi',  SelectKBest(chi2, k=15)),
                     ('clf', RandomForestClassifier())])

In [ ]:
model = pipeline.fit(X_train, y_train)
with open('Churn_chi15_outdata.pickle', 'wb') as f:
    pickle.dump(model, f)
ytest = np.array(y_test)
# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
threshold = 0.1

predicted_proba = model.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
accuracy

print(classification_report(ytest, predicted))
print(confusion_matrix(ytest, predicted))

In [ ]:
threshold = 0.4

predicted = model.predict_proba(X_test)
predicted [:,0] = (predicted [:,0] < threshold).astype('int')
predicted [:,1] = (predicted [:,1] >= threshold).astype('int')


accuracy = accuracy_score(y_test, predicted)
print(round(accuracy,4,)*100, "%")

In [ ]:
#15
  precision    recall  f1-score   support

           0       0.93      0.98      0.95     11123
           1       0.67      0.32      0.44      1267

    accuracy                           0.91     12390
   macro avg       0.80      0.65      0.69     12390
weighted avg       0.90      0.91      0.90     12390

[[10923   200]
 [  859   408]]

#4
   precision    recall  f1-score   support

           0       0.91      0.99      0.95     11123
           1       0.54      0.13      0.20      1267

    accuracy                           0.90     12390
   macro avg       0.72      0.56      0.58     12390
weighted avg       0.87      0.90      0.87     12390

[[10986   137]
 [ 1107   160]]
#5
 precision    recall  f1-score   support

           0       0.93      0.97      0.95     11123
           1       0.60      0.40      0.48      1267

    accuracy                           0.91     12390
   macro avg       0.77      0.69      0.72     12390
weighted avg       0.90      0.91      0.90     12390

[[10780   343]
 [  755   512]]
#6
 precision    recall  f1-score   support

           0       0.91      0.98      0.94     11123
           1       0.47      0.15      0.23      1267

    accuracy                           0.90     12390
   macro avg       0.69      0.56      0.59     12390
weighted avg       0.87      0.90      0.87     12390


In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
model['chi']

In [ ]:
scores = pd.DataFrame(-np.log10(model['chi'].pvalues_))

In [ ]:
predictors=pd.DataFrame(X_train.columns)

In [ ]:
##change didn't renew to null -- Did not renew ex contacrt 3777541929
## doesn't make sence to have not renewal  as a valid entry for past renewal

In [ ]:
#Sub-Batch_Type_WINBACK=86.72553029, last_renewal_Did not renew=108.92708617

In [ ]:
col_res=pd.concat([predictors,scores], axis = 1)

In [ ]:
col_res.columns=['Variable','Score']

In [ ]:
col_res.sort_values('Score', inplace=True, ascending=False)

In [ ]:
col_res.head(20)

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(range(len(predictors)), scores)
plt.xticks(range(len(predictors)), predictors, rotation='vertical')
plt.show()

In [ ]:
model['chi'].shape()

In [ ]:
model['chi'].pvalues_

In [ ]:
model['chi'].scores_

In [ ]:
model['clf'].feature_importances_

In [ ]:
list(zip(train[features], model['clf'].feature_importances_))

In [ ]:

rf = RandomForestClassifier()
perm = PermutationImportance(model['clf'], cv = None, refit = False, n_iter = 50).fit(X_train, y_train)
perm_imp_eli5 = imp_df(X_train.columns, perm.feature_importances_)